<a href="https://colab.research.google.com/github/exp0nent/Kaggle_Practice/blob/main/NLP_Emotion_Project/emotion_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aatirali18","key":"e757b143d56bf410878f957265b2a3ab"}'}

In [3]:
!mkdir -p /root/.kaggle
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d praveengovi/emotions-dataset-for-nlp

Dataset URL: https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp
License(s): CC-BY-SA-4.0
  0% 0.00/721k [00:00<?, ?B/s]
100% 721k/721k [00:00<00:00, 252MB/s]


In [5]:
!ls

emotions-dataset-for-nlp.zip  kaggle.json  sample_data


In [6]:
!unzip emotions-dataset-for-nlp.zip

Archive:  emotions-dataset-for-nlp.zip
  inflating: test.txt                
  inflating: train.txt               
  inflating: val.txt                 


In [7]:
!ls

emotions-dataset-for-nlp.zip  sample_data  train.txt
kaggle.json		      test.txt	   val.txt


In [8]:
import pandas as pd

In [9]:
def load_emotion_txt(file_path):
    texts = []
    labels = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if ';' in line:
                text, label = line.rsplit(';', 1)
                texts.append(text.strip())
                labels.append(label.strip())

    return pd.DataFrame({
        'text': texts,
        'emotion': labels
    })


In [10]:
train_df = load_emotion_txt("train.txt")
val_df   = load_emotion_txt("val.txt")
test_df  = load_emotion_txt("test.txt")

In [11]:
train_df.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [12]:
train_df.info()
train_df['emotion'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     16000 non-null  object
 1   emotion  16000 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


,count
emotion,
joy,5362
sadness,4666
anger,2159
fear,1937
love,1304
surprise,572


In [13]:
train_df.isnull().sum()

,0
text,0
emotion,0


In [14]:
train_df['emotion'].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [16]:
unique_emotions = train_df['emotion'].unique() # Define mapping based on training data
emotion_numbers = {}
for i, emo in enumerate(unique_emotions):
    emotion_numbers[emo] = i

train_df['emotion_encoded'] = train_df['emotion'].map(emotion_numbers)
val_df['emotion_encoded'] = val_df['emotion'].map(emotion_numbers)
test_df['emotion_encoded'] = test_df['emotion'].map(emotion_numbers)

print("Emotion to Numerical Mapping:")
for emotion, number in emotion_numbers.items():
    print(f"{emotion}: {number}")

print("\nTrain DataFrame with encoded emotions:")
display(train_df.head())
print("\nValidation DataFrame with encoded emotions:")
display(val_df.head())
print("\nTest DataFrame with encoded emotions:")
display(test_df.head())

Emotion to Numerical Mapping:
sadness: 0
anger: 1
love: 2
surprise: 3
fear: 4
joy: 5

Train DataFrame with encoded emotions:


,text,emotion,emotion_encoded
0,i didnt feel humiliated,sadness,0
1,i can go from feeling so hopeless to so damned...,sadness,0
2,im grabbing a minute to post i feel greedy wrong,anger,1
3,i am ever feeling nostalgic about the fireplac...,love,2
4,i am feeling grouchy,anger,1



Validation DataFrame with encoded emotions:


,text,emotion,emotion_encoded
0,im feeling quite sad and sorry for myself but ...,sadness,0
1,i feel like i am still looking at a blank canv...,sadness,0
2,i feel like a faithful servant,love,2
3,i am just feeling cranky and blue,anger,1
4,i can have for a treat or if i am feeling festive,joy,5



Test DataFrame with encoded emotions:


,text,emotion,emotion_encoded
0,im feeling rather rotten so im not very ambiti...,sadness,0
1,im updating my blog because i feel shitty,sadness,0
2,i never make her separate from me because i do...,sadness,0
3,i left with my bouquet of red and yellow tulip...,joy,5
4,i was feeling a little vain when i did this one,sadness,0


In [19]:
train_df['text'] = train_df['text'].apply(lambda x : x.lower())
val_df['text'] = val_df['text'].apply(lambda x : x.lower())
test_df['text'] = test_df['text'].apply(lambda x : x.lower())

In [22]:
import string
def remove_punc(text):
    return text.translate(str.maketrans(' ', ' ', string.punctuation))

train_df['text'] = train_df['text'].apply(remove_punc)
val_df['text'] = val_df['text'].apply(remove_punc)
test_df['text'] = test_df['text'].apply(remove_punc)

In [24]:
def remove_no(txt):
    new_text = ""
    for char in txt:
        if not char.isdigit():
            new_text += char
    return new_text.strip()

train_df['text'] = train_df['text'].apply(remove_no)
val_df['text'] = val_df['text'].apply(remove_no)
test_df['text'] = test_df['text'].apply(remove_no)

print("Text data after removing numbers:")
display(train_df.head())

Text data after removing numbers:


,text,emotion,emotion_encoded
0,i didnt feel humiliated,sadness,0
1,i can go from feeling so hopeless to so damned...,sadness,0
2,im grabbing a minute to post i feel greedy wrong,anger,1
3,i am ever feeling nostalgic about the fireplac...,love,2
4,i am feeling grouchy,anger,1


In [25]:
def remove_emojis(txt):
    new_text = ""
    for char in txt:
        if char.isascii():
            new_text += char
    return new_text.strip()

train_df['text'] = train_df['text'].apply(remove_emojis)
val_df['text'] = val_df['text'].apply(remove_emojis)
test_df['text'] = test_df['text'].apply(remove_emojis)

print("Text data after removing emojis:")
display(train_df.head())

Text data after removing emojis:


,text,emotion,emotion_encoded
0,i didnt feel humiliated,sadness,0
1,i can go from feeling so hopeless to so damned...,sadness,0
2,im grabbing a minute to post i feel greedy wrong,anger,1
3,i am ever feeling nostalgic about the fireplac...,love,2
4,i am feeling grouchy,anger,1


In [26]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [27]:
!git clone https://github.com/exp0nent/Kaggle_Practice.git

Cloning into 'Kaggle_Practice'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 124 (delta 61), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (124/124), 4.08 MiB | 9.74 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [47]:
!mkdir NLP_Emotion_Project

mkdir: cannot create directory ‘NLP_Emotion_Project’: File exists


In [48]:
"""!git clone https://github.com/exp0nent/Kaggle_Practice.git
%cd Kaggle_Practice
!mkdir NLP_Emotion_Project"""


'!git clone https://github.com/exp0nent/Kaggle_Practice.git\n%cd Kaggle_Practice\n!mkdir NLP_Emotion_Project'

In [49]:
"""(Git commands – professional way)
%cd Kaggle_Practice
!git status
!git add NLP_Emotion_Project/emotion_nlp.ipynb
!git commit -m "Updated emotion NLP notebook"
!git push"""

'(Git commands – professional way)\n%cd Kaggle_Practice\n!git status\n!git add NLP_Emotion_Project/emotion_nlp.ipynb\n!git commit -m "Updated emotion NLP notebook"\n!git push'

In [50]:
%cd Kaggle_Practice

[Errno 2] No such file or directory: 'Kaggle_Practice'
/content/Kaggle_Practice


In [51]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [34]:
!git add NLP_Emotion_Project/emotion_nlp.ipynb

fatal: pathspec 'NLP_Emotion_Project/emotion_nlp.ipynb' did not match any files


Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@a06f983a6d4c.(none)')


In [42]:
!git config --global user.email "aatirali18@gmail.com"
!git config --global user.name "Aatir Ali"

In [43]:
!git commit -m "Updated emotion NLP notebook"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [45]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address
